## J.C Hull: Options, Futures, And Other Derivatives

### Chapters 19 - 25

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

r = 0.01
S = 30
K = 40
T = 240/365
sigma = 0.30


## Chapter 19 The Greek Letters
(Below are calculators for Euro options on underlying stocks that pay no dividends)

### Stop Loss Strategy
consider an institution that has written a call option.... hold a naked position whenever the stock price is less than K and a covered position whenever the stock price is greater than K. The procedure is designed to ensure that at time T the institution owns the stock if the option closes in the money and does not own it if the option closes out of the money

A stop-loss strategy, although superficially attractive, does not work particularly well as a hedging procedure. Consider its use for an out-of-the-money option. If the stock price never reaches the strike price K, the hedging procedure costs nothing. If the path of the stock price crosses the strike price level many times, the procedure is quite expensive.

### Delta Hedging 
The rate of change of the option price with respect to the price of the underlying asset.

Suppose that the delta of a call option on a stock is 0.6. This means that when the stock price changes by a small amount, the option price changes by about 60% of that amount.

the delta of an option does not remain constant, the trader’s position remains delta hedged (or delta neutral) for only a relatively short period of time. 



In [5]:
# Delta calculator for European option long position

def delt(S, K, T, r, sigma, opttype ="C"):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    
    if opttype == "C":
        delta = norm.cdf(d1)
    elif opttype =="P":
        delta = norm.cdf(d1) - 1 
    return delta

#### Euro Calls
Using delta hedging for a short position in a European call option involves maintaining a long position of $\frac{\partial c}{\partial S}$ for each option sold. delta hedging for a long position in a European call option involves maintaining a short position of $\frac{\partial c}{\partial S}$ shares for each option purchased.

#### Euro Puts
a long position in a put option should be hedged with a long position in the underlying stock, and a short position in a put option should be hedged with a short position in the underlying stock




### Theta
The rate of change of the value of the portfolio with respect to the passage of time with all else remaining the same. Theta is sometimes referred to as the time decay of the portfolio


In [4]:
def Theta(S, K, T, r, sigma, opttype ="C"):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - (sigma * np.sqrt(T))
    
    if opttype == "C":
        theta = -S*norm.pdf(d1)*sigma/(2*np.sqrt(T)) - r*K*np/exp(-r*T)*norm.cdf(d2)
    elif opttype =="P":
        theta = -S*norm.pdf(d1)*sigma/(2*np.sqrt(T)) + r*K*np/exp(-r*T)*norm.cdf(-d2)
    return theta/365 ## Scaled to days in a year
    

Theta is usually negative for an option. This is because, as time passes with all else remaining the same, the option tends to become less valuable

Theta is not the same type of hedge parameter as delta. There is uncertainty about the future stock price, but there is no uncertainty about the passage of time. It makes sense to hedge against changes in the price of the underlying asset, but it does not make any sense to hedge against the passage of time.

However, Theta is a proxy for gamma.



### Gamma

The gamma of a portfolio of options on an underlying asset is the rate of  change of the portfolio’s delta with respect to the price of the underlying asset. It is the second partial derivative of the portfolio with respect to asset price:
$$\frac{\partial^2 v}{\partial S^{2}}$$

If gamma is small, delta changes slowly, and adjustments to keep a portfolio delta neutral need to be made only relatively infrequently. However, if gamma is highly negative or highly positive, delta is very sensitive to the price of the underlying asset. It is then quite risky to leave a delta-neutral portfolio unchanged for any length of time

Approximately, the change in the value of a portfolio is given by,
$$\Delta Port = \frac{\partial v}{\partial S} \Delta t + \frac{1}{2} \frac{\partial^2 v}{\partial S^{2}} \Delta S^2$$

#### Making a Portfolio Gamma Neutral
A position in the underlying asset has zero gamma and cannot be used to change the gamma of a portfolio. What is required is a position in an instrument such as an option that is not linearly dependent on the underlying asset.

Suppose that a delta-neutral portfolio has a gamma equal to $G_{P}$, and a traded option has a gamma equal to G. If the number of traded options added to the portfolio is wT , the gamma of the portfolio is
$$w_{T}G_{P} + G$$
Hence, the position in the traded option necessary to make the portfolio gamma neutral is $-G/G_{P}$



Delta neutrality provides protection against relatively small stock price moves between rebalancing. Gamma neutrality provides protection against larger movements in this stock price between hedge rebalancing




## Vega
In practice, volatilities change over time. This means that the value of a derivative is liable to change because of movements in volatility as well as because of changes in the asset price and the passage of time.

The vega of a portfolio of derivatives, V, is the rate of change of the value of the portfolio with respect to the volatility of the underlying asset.

#####  If vega is highly positive or highly negative, the portfolio’s value is very sensitive to small changes in volatility. If it is close to zero, volatility changes have relatively little impact on the value of the portfolio.

vega of a portfolio can be changed, similarly to the way gamma can be changed, by adding a position in a traded option. 

a portfolio that is gamma neutral will not in general be vega neutral,
and vice versa. If a hedger requires a portfolio to be both gamma and vega neutral, at least two traded derivatives dependent on the underlying asset must usually be used.

Vega neutrality protects against a variable volatility. As might be
expected, whether it is best to use an available traded option for vega or gamma hedging depends on the time between hedge rebalancing and the volatility of the volatility.

When volatilities change, the implied volatilities of short-dated options tend to change by more than the implied volatilities of long-dated options. The vega of a portfolio is therefore often calculated by changing the volatilities of long-dated options by less than that of short-dated options




In [ ]:
def vega(r, S, K, T, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - (sigma * np.sqrt(T))
    vega = S*norm.pdf(d1)*np.sqrt(T)
    
    return vega*0.01 ## To scale to % changes

## Rho
The rho of a portfolio of options is the rate of change of the value of the portfolio with respect to the interest rate. It measures the sensitivity of the value of a portfolio to a change in the interest rate when all else remains the same. 


In [ ]:
def Rho(S, K, T, r, sigma, opttype ="C"):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - (sigma * np.sqrt(T))
    
    if opttype == "C":
        rho = K*T*np.exp(-r*T)*norm.cdf(d2)
    elif opttype =="P":
        rho = -K*T*np.exp(-r*T)*norm.cdf(-d2)
    
    return rho

## Chapter 20 Volatility Smiles

the implied volatility of a European call option is the same as that of a European put option when they have the same strike price and time to maturity. This means that the volatility smile for European calls with a certain maturity is the same as that for European puts with the same maturity

put–call parity provides a relationship between the prices of European call and put options when they have the same strike price and time to maturity. With a dividend yield on the underlying asset of q, the relationship is,
$$p + S_{0}e^{-qT}=c + Ke^{-rT}$$
It does not require any assumption about the probability distribution of the asset price in the future. It is true both when the asset price distribution is lognormal and when it is not lognormal.

Suppose that for a particular value of volatility, we have option prices given by the BSM model. Suppose further that we also have these options in the market with their respective market prices, then by put call parity for both of the pairs of options,
$$p_{BS} + S_{0}e^{-qT}=c_{BS} + Ke^{-rT}$$
$$p_{mkt} + S_{0}e^{-qT}=c_{mkt} + Ke^{-rT}$$
Then, subtracting these two equations, we obtain
$$p_{BS}-p_{mkt} = c_{BS}-c_{mkt}$$

This shows that the dollar pricing error when the Black–Scholes–Merton model is used to price a European put option should be exactly the same as the dollar pricing error when it is used to price a European call option with the same strike price and time to maturity

##### , for a given strike price and maturity, the correct volatility to use in conjunction with the Black–Scholes–Merton model to price a European call should always be the same as that used to price a European put. This means that the volatility smile (i.e., the relationship between implied volatility and strike price for a particular maturity) is the same for European calls and European puts. More generally, it means that the volatility surface (i.e., the implied volatility as a function of strike price and time to maturity) is the same for European calls and European puts. These results are also true to a good approximation for American options.

### Reasons for the volatility Smile (Foreign Currency)
1. Volatility of the asset is not constant
2. Price of the asset doesn't always change smoothly with no jumps

The volatility of an exchange rate is far from constant, and exchange rates frequently exhibit jumps.3 It turns out that the effect of both a nonconstant volatility and jumps is that extreme outcomes become more likely

The impact of jumps and nonconstant volatility depends on the option maturity. As the maturity of the option is increased, the percentage impact of a nonconstant volatility on prices becomes more pronounced, but its percentage impact on implied volatility usually becomes less pronounced. The percentage impact of jumps on both prices and the implied volatility becomes less pronounced as the maturity of the option is increased. the volatility smile becomes less pronounced as option maturity increases.

#### Equity Options


#### Other ways to characterise Volatility Smile
when the equity price increases, the volatility skew tends to
move to the right, and when the equity price decreases, it tends to move to the left.

For this reason the volatility smile is often calculated as the relationship between the implied
volatility and K/$S_{0}$

A refinement of this is to calculate the volatility smile as the relationship between the
implied volatility and K/$F_{0}$. The argument for this is that $F_{0}$, not $S_{0}$, is the expected stock price on the option’s maturity date in a risk-neutral world

Another approach to defining the volatility smile is as the relationship between the
implied volatility and the delta of the option

#### Volatility Surfaces 
(Time to Maturity and K/$F_{0}$)

Traders allow the implied volatility to depend on time to maturity as well as strike price. Implied volatility tends to be an increasing function of maturity when short-dated volatilities are historically low. This is because there is then an expectation that volatilities will increase. Similarly, volatility tends to be a decreasing function of maturity when short-dated volatilities are historically high. This is because there is then an expectation that volatilities will decrease.

#### Greek Letters
When accounting for nonconstant volatility, the Greek's are no longer correct. Assuming  that the
relationship between the implied volatility and K/S for an option with a certain time, Delta is now
$$Delta = \frac{\partial C_{BS}}{\partial S} + \frac{\partial C_{BS}}{\partial \sigma_{imp}}\frac{\partial \sigma_{imp}}{\partial S}$$

As a result, delta is higher than that given by the Black–Scholes–Merton assumptions

Models have most effect on the pricing of derivatives when similar derivatives do not
trade actively in the market. For example, the pricing of many of the nonstandard
exotic derivatives we will discuss in later chapters is model-dependent.

#### Determining Implied Risk Neutral Distributions from Volatility Smiles
(Breeden and Litzenberger 1978)
The price of a European call option on an asset with strike price K and maturity T is
given by,
$$c = e^{-rT}\int_{S_{T}=K}^{\infty}(S_{T}-K)g(S_{T})dS_{T}$$

g is the risk-neutral probability density function of $S_{T]$. After differentiating twice with respect to K will give the probability density function for g,

$$g(K)=e^{rT}\frac{\partial^2 C}{\partial K^2}$$



## Chapter 22 Value at Risk
"I am X percent certain there will not be a loss of more than V dollars in the next N days."

The variable V is the VaR of the portfolio. It is a function of two parameters: the time
horizon (N days) and the confidence level (X%). It is the loss level over N days that has
a probability of only (100 - X)% of being exceeded.

### Time Horizon
$$Nday\;VaR=1Day\;VaR\sqrt{N}$$


### Daily Volatilities
$$\sigma_{year} = \sigma_{day}\sqrt{252}$$


### Single Asset Case
Assume a position of 10 million in shares of company B, and the the volatility is 2% per day, then the standard deviation of daily changes in the value of the position of 10 million is 2% * 10 million = 200,000

Then assuming the change in portfolio value is normally distributed, we can apply a 99% confidence interval (1-99% = 1%). $N^{-1}(0.01)$=2.326 standard deviations. This means there is a 1% probability that a normally distributed variable will decrease in value more than 2.326 standard deviations.

This is also equivalent to being 99% certain that a normally distributed variable will not decrease in value by more than 2.326 standard deviations

In all, we can conclude that 
##### "I am 99 percent certain there will not be a loss of more than 2.326 * 200,000 = 465,300 dollars in the next 1 days."

### Two - Asset Case
The example above only for works for a single asset. For multiple assets that are correlated, we then need to use the variance formula for a portfolio containing n amount of risky assets

### Benefits of Diversification
Standard result of modern portfolio theory that including assets that less than perfectly correlated leads to risk by diversified away

### Linear Model
The examples above are simplified illustrations of a linear model for calculating VaR


### Linear Model and Options
Let delta be defined by the following,
$$\delta = \frac{\Delta P}{\Delta S}$$
Where $\Delta P$ is the dollar change in the portfolio in 1 day, $\Delta S$ is the dollar change in stock price in 1 day
$$\Delta x = \frac{\Delta S}{S}$$

Then we can now define several positions that includes options
$$\Delta P = \Sigma_{i=1}^{n} S_{i}\delta_{i}\Delta x_{i}$$

### The Quadratic Model
##### When a portfolio includes options, the linear model is an approximation. It does not take account of the gamma of the portfolio. As discussed in Chapter 19, delta is defined as the rate of change of the portfolio value with respect to an underlying market variable and gamma is defined as the rate of change of the delta with respect to the market variable. 

##### Gamma measures the curvature of the relationship between the portfolio value and an underlying market variable.



### Principal Component Analysis


## Chapter 23 Estimating Volatilities and Correlations

### Estimating Volatility (Unbiased Estimates)
Let $u_{i}$ be the continuously compounded return during day i
$$u_{i}=ln\frac{S_{i}}{S_{i-1}}$$
Then the unbiased estimate of the variance rate per day,
$$\sigma^{2}_{n} = \frac{1}{m-1}\Sigma_{i=1}^{m} (u_{i-1}-\bar{u})^2$$

(Remember that weighting schemes, it makes sense to give more weight to recent data, thus more recent volatility is more important)

### Exponentially Weighted Moving Average Model
The weights decrease exponentially as you move back in time, $\lambda$ being a constant between 0 and 1
$$\sigma^{2}_{n} = (1-\lambda)\Sigma_{i=1}^{m}\lambda^{i-1}u_{i-1}^{2}+\lambda^m \sigma^{2}_{n-m}$$

(The EWMA is a particular case of the GARCH model)

### GARCH(1,1) Model
$$\sigma^{2}_{n}=\gamma V_{L} + \alpha u_{i-1}^{2} + \beta \sigma^{2}_{n-1}$$
Where $V_{L}$ is the long run average variance rate, and
$$\gamma + \alpha + \beta = 1 $$

The long run average variance rate, $V_{L}$, is usually derived from estimated daily data.

##### Note that  $\alpha + \beta$ must be <1 for a stable GARCH process

### Mean Reversion
The GARCH (1,1) model recognizes that over time the variance tends to get pulled back to a long-run average level



### Maximum Likelihood Methods
We are assuming the random variable follows a normal distribution, then we can use standard results for MLE to obtain best estimates for parameters

This means we can estimate the best parameters in the models above.

### How Good is the Model
(Below is standard information for time series modelling)

The assumption underlying a GARCH model is that volatility changes with the passage of time. During some periods volatility is relatively high; during other periods it is relatively low

We can test how true this is by examining the autocorrelation structure by using a Ljung - Box Statistic


### Using GARCH(1,1) to forecast future Volatility

$$E[\sigma^{2}_{n+t}]=V_{L}+(\alpha +\beta )^{t}(\sigma^{2}_{n}-V_{L})$$

### Volatility Term Structures
Define
$$V(t) = E[sigma^{2}_{n+t}]$$
and
$$a=ln\frac{1}{\alpha + \beta}$$
Then the equation for forecasting future volatility,
$$E[\sigma^{2}_{n+t}]=V_{L}+(\alpha +\beta )^{t}(\sigma^{2}_{n}-V_{L})$$
 becomes,
$$V(t) =V_{L}+e^{-at}[V(0)-V_{L}]$$

This is an estimate of the instantaneous variance rate in t days.
The average variance rate per day between today and time T is given by

$$\frac{1}{T} \int_{0}^{T} V(t)dt=V_{L}+\frac{1-e^{-at}}{aT}[V(0)-V_{L}]$$

Then we can now use this to find the volatility per annum that should be used to price to a T-day option under GARCH(1,1). Assuming 252 trading days per year,

$$\sigma^2 (T)=252[V_{L}+\frac{1-e^{-at}}{aT}[V(0)-V_{L}]]$$

This is the relationship between the implied volatilities of the options and their maturities

market prices of different options on the same asset are often used to calculate a volatility term structure

### Impact of Volatility Changes
The above equation then can also be used to see the impact of a change in Instantaneous volatilities on T-day option





## Chapter 24 Credit Risk


### Hazard Rates
We will refer to this as the unconditional default probability. It is the probability of default during the t year. I.e, The probability that it will default during the t year conditional on no earlier default 

Let V(t) be the cumulative probability of the company surviving to time t,
then setting the probability equal to $\lambda (t)\Delta t$, we have that

$$\frac{dV(t)}{dt}=-\lambda (t)V(t)$$
With solution,

$$V(t)=exp(-\int_{0}^{t}\lambda (\tau )d\tau)$$


Defining $Q(t)$ as the probability of default by time t,
$$Q(t)=1-V(t)$$
$$Q(t)=1-exp(-\int_{0}^{t}\lambda (\tau )d\tau)$$


### Estimating Default Probabilities From Bond Yield Spreads
A bond’s yield spread is the excess of the promised yield on the bond over the risk-free rate. The usual assumption is that the excess yield is compensation for the possibility of default.

### Real-World vs. Risk-Neutral Probabilities
bonds do not default independently of each other. There are periods of time when default rates are very low and periods of time when they are very high.

Bond returns are highly skewed with limited upside, risk is difficult to diversify away.

In practice, many bond portfolios are far from fully diversified. As a result, bond traders may earn an extra return for bearing nonsystematic risk as well as for bearing the systematic risk

##### When valuing credit derivatives or estimating the impact of default risk on the pricing of instruments, risk-neutral default probabilities should be used

##### When carrying out scenario analyses to calculate potential future losses from defaults, real-world default probabilities should be used.

### Using Equity Prices to Estimate Default Probabilities

to estimate a company’s real-world probability of default, we are relying on the company’s credit rating. Unfortunately, credit ratings are revised relatively infrequently. This has led some analysts to argue that equity prices
can provide more up-to-date information for estimating default probabilities.


## Chapter 25 Credit Derivatives


### Credit Default Swaps

It is a contract that provides insurance against the risk of a default by
particular company. The company is known as the reference entity and a default by the company is known as a credit event. The buyer of the insurance obtains the right to sell bonds issued by the company for their face value when a credit event occurs and the seller of the insurance agrees to buy the bonds for their face value when a credit event occurs.

The total face value of the bonds that can be sold is known as the credit default swap’s notional principal

The buyer of the CDS makes periodic payments to the seller until the end of the life of the CDS or until a credit event occurs.

The total amount paid per year, as a percent of the notional principal, to buy
protection is known as the CDS spread

Usually a credit event is defined as a failure to make a payment as it becomes due, a restructuring of debt, or a bankruptcy.

### Credit Default Swaps and Bond Yields

CDS–bond basis = CDS spread  Bond yield spread

This shows that the spread of the yield on an n-year bond issued by a company over the risk-free rate should approximately equal the company’s n-year CDS spread.

The arbitrage argument given above suggests that the CDS–bond basis should be
close to zero. 

### Estimating Default Probabilities
The default probabilities used to value a CDS should be risk-neutral default probabilities, not real-world default probabilities

Risk-neutral default probabilities can be estimated from
bond prices or asset swaps

An alternative is to imply them from CDS quotes. The latter approach is similar to the practice in options markets of implying volatilities from the prices of actively traded options and using them to value other options.

### CDS Forwards and Options

A forward credit default swap is the obligation to buy or sell a particular credit default swap on a particular reference entity at a particular future time T. If the reference entity defaults before time T, the forward contract ceases to exist.

A credit default swap option is an option to buy or sell a particular credit default swap on a particular reference entity at a particular future time T.


